In [3]:
import random
import os
from datetime import datetime, timedelta

def generar_archivos_meteoros():
    # la semilla define todo el conjunto de datos
    # mismo numero = mismos archivos siempre
    # funciona igual que las semillas de minecraft, es decir, definen una combinación númerica aleatoria pero es unica
    # la semilla 1 es distinta a la 2, es como un codigo de barras que los identifica
    semilla_actual = 122025
    random.seed(semilla_actual)

    # definimos la ventana de tiempo de 2 dias como pide la tarea
    # desde el 1 de diciembre 2025 a las 2am hasta el 3 de diciembre 2025 a las 2am
    tiempo_inicio = datetime(2025, 12, 1, 2, 0, 0)
    tiempo_final = datetime(2025, 12, 3, 2, 0, 0)

    # calculamos cuantos segundos hay en total en 2 dias
    # esto nos sirve para distribuir los eventos en el tiempo
    duracion_total_segundos = int((tiempo_final - tiempo_inicio).total_seconds())

    # generamos un numero aleatorio de archivos entre 500 y 999
    # como especifica la tarea, debe ser entre esos valores
    cantidad_eventos = random.randint(500, 999)
    print(f"Generando {cantidad_eventos} archivos de meteoros.")

    # creamos la carpeta donde se guardaran los archivos si no existe
    if not os.path.exists("archivos_meteoros"):
        os.makedirs("archivos_meteoros")

    # creamos una lista con todos los segundos disponibles en los 2 dias
    # luego elegimos aleatoriamente algunos segundos para los eventos
    # random.sample asegura que no haya segundos repetidos
    segundos_disponibles = list(range(duracion_total_segundos))
    instantes_seleccionados = random.sample(segundos_disponibles, cantidad_eventos)

    # ordenamos los segundos de menor a mayor para que los eventos
    # esten en orden cronologico como pide la tarea
    instantes_seleccionados.sort()

    # ahora creamos cada archivo individualmente
    # usamos enumerate para tener un contador i que empieza en 0
    for i, segundos in enumerate(instantes_seleccionados):
        # calculamos el momento exacto del evento sumando los segundos al inicio
        tiempo_evento = tiempo_inicio + timedelta(seconds=segundos)

        # generamos los datos del meteoro segun los rangos especificados
        # duracion entre 0.1 y 5.0 segundos con 1 decimal
        duracion = round(random.uniform(0.1, 5.0), 1)

        # altura sobre el horizonte entre 10.0 y 90.0 grados
        altura = round(random.uniform(10.0, 90.0), 1)

        # acimut (direccion en el horizonte) entre 0.0 y 359.0 grados
        acimut = round(random.uniform(0.0, 359.0), 1)

        # formateamos el numero del archivo para que tenga 3 digitos
        # 1 se convierte en 001, 2 en 002, etc hasta 999
        numero_archivo = f"{i+1:03d}"

        # creamos el nombre del archivo con el formato requerido
        nombre_archivo = f"archivos_meteoros/meteoro_{numero_archivo}.txt"

        # abrimos el archivo en modo escritura y guardamos los datos
        with open(nombre_archivo, 'w') as archivo:
            # formateamos la linea con todos los datos separados por comas
            # fecha en formato YYYY-MM-DD, hora en HH:MM:SS
            linea = f"{tiempo_evento.strftime('%Y-%m-%d')}, {tiempo_evento.strftime('%H:%M:%S')}, {duracion}, {altura}, {acimut}"
            archivo.write(linea)

    # mensaje final confirmando que todo salio bien
    print(f"Se crearon {cantidad_eventos} archivos.")
    print(f"Estos archivos fueron generados por la seed {semilla_actual}.")

if __name__ == "__main__":
    generar_archivos_meteoros()

Generando 921 archivos de meteoros.
Se crearon 921 archivos.
Estos archivos fueron generados por la seed 122025.


In [8]:
import os
from datetime import datetime

def analizar_eventos_meteoros():

    eventos = []
    directorio = "archivos_meteoros"

    # primero verificamos que exista la carpeta con los archivos
    # si no existe, significa que no se generaron los datos todavia
    if not os.path.exists(directorio):
        print(f"error: no se encuentra el directorio '{directorio}'")
        print("ejecuta primero: python generate_meteors.py")
        return

    # empezamos a leer todos los archivos de la carpeta

    archivos_procesados = 0
    # recorremos cada archivo en la carpeta
    for nombre_archivo in os.listdir(directorio):
        # solo nos interesan los archivos que empiezan con 'meteoro_' y terminan con '.txt'
        if nombre_archivo.startswith("meteoro_") and nombre_archivo.endswith(".txt"):
            ruta_archivo = os.path.join(directorio, nombre_archivo)

            try:
                # abrimos cada archivo y leemos su contenido
                with open(ruta_archivo, 'r') as archivo:
                    linea_datos = archivo.readline().strip()
                    # separamos los datos por comas y limpiamos espacios
                    componentes = [comp.strip() for comp in linea_datos.split(",")]

                    # verificamos que tenga exactamente 5 partes
                    # fecha, hora, duracion, altura, acimut
                    if len(componentes) != 5:
                        continue  # si no tiene 5, saltamos este archivo

                    # convertimos la fecha y hora a un objeto datetime
                    # esto nos permite hacer calculos de tiempo
                    fecha_hora = datetime.strptime(
                        f"{componentes[0]} {componentes[1]}",
                        "%Y-%m-%d %H:%M:%S"
                    )

                    # creamos un diccionario con todos los datos del evento
                    evento = {
                        'timestamp': fecha_hora,           # fecha y hora del evento
                        'duracion_seg': float(componentes[2]),  # duracion en segundos
                        'altura_grados': float(componentes[3]), # altura en grados
                        'acimut_grados': float(componentes[4])  # acimut en grados
                    }
                    eventos.append(evento)
                    archivos_procesados += 1  # contamos cuantos archivos procesamos

            except Exception as e:
                # si hay algun error al procesar un archivo, mostramos el error
                # pero continuamos con los demas archivos
                print(f"error procesando {nombre_archivo}: {e}")
                continue

    # ordenamos todos los eventos por fecha y hora
    # esto es importante para calcular los intervalos correctamente
    eventos_ordenados = sorted(eventos, key=lambda x: x['timestamp'])
    total_eventos = len(eventos_ordenados)

    print(f"Se procesaron {archivos_procesados} archivos")
    print(f"{total_eventos} eventos ordenados temporalmente")

    # calculamos los intervalos entre eventos
    if total_eventos > 1:
        diferencias_temporales = []

        # recorremos los eventos ordenados y calculamos la diferencia
        # entre cada evento y el anterior
        for i in range(1, total_eventos):
            # calculamos la diferencia en segundos entre eventos consecutivos
            intervalo = (eventos_ordenados[i]['timestamp'] -
                        eventos_ordenados[i-1]['timestamp']).total_seconds()
            diferencias_temporales.append(intervalo)

        # calculamos el promedio de todos los intervalos
        # sumamos todos los intervalos y dividimos por la cantidad
        intervalo_promedio = sum(diferencias_temporales) / len(diferencias_temporales)

        # guardamos los resultados en el archivo stats.txt
        # con el formato que pide la tarea
        with open("stats.txt", 'w') as archivo_salida:
            archivo_salida.write(f"Total events: {total_eventos}\n")
            archivo_salida.write(f"Average time between meteors: {intervalo_promedio:.1f} seconds\n")

        print("Analisis completado")
        print(f"Resultados guardados en 'stats.txt'")
        print(f"Resumen: {total_eventos} eventos, intervalo promedio: {intervalo_promedio:.1f} s")

    else:
        print("error: No hay suficientes eventos para analisis.")

# esto hace que cuando ejecutamos el archivo directamente
# se llame a la funcion de analisis
if __name__ == "__main__":
    analizar_eventos_meteoros()

Se procesaron 921 archivos
921 eventos ordenados temporalmente
Analisis completado
Resultados guardados en 'stats.txt'
Resumen: 921 eventos, intervalo promedio: 187.6 s
